In [1]:
import arcpy

arcpy.env.workspace = r"C:\Gis class\Homework4\HWK4_output.gdb"
arcpy.env.overwriteOutput = True

print("Workspace ready")


Workspace ready


In [2]:
# Path to CSV
csv_file = r"C:\Gis class\Homework4\garages.csv"

# Output feature class name
garage_points = "GaragePoints"

# Convert CSV to Points
arcpy.management.XYTableToPoint(
    csv_file,
    garage_points,
    "X",   # X field name
    "Y",   # Y field name
    coordinate_system=arcpy.SpatialReference(4326)
)

print("Garage points created")


Garage points created


In [3]:
# Project GaragePoints (WGS84 degrees) to UTM (meters)
garage_points_proj = "GaragePoints_UTM14N"

arcpy.management.Project(
    in_dataset="GaragePoints",
    out_dataset=garage_points_proj,
    out_coor_system=arcpy.SpatialReference(32614)  # WGS 84 / UTM zone 14N (College Station)
)

print("Projected to UTM (meters):", garage_points_proj)


Projected to UTM (meters): GaragePoints_UTM14N


In [4]:
# Buffer 150 meters
garage_buffer = "GaragePoints_Buffer150m"

arcpy.analysis.Buffer(
    in_features="GaragePoints_UTM14N",
    out_feature_class=garage_buffer,
    buffer_distance_or_field="150 Meters",
    dissolve_option="NONE"
)

print("Buffer created:", garage_buffer)


Buffer created: GaragePoints_Buffer150m


In [5]:
# Intersect buffer with buildings (Structures)
intersection_output = "GarageBuildingIntersection"

arcpy.analysis.Intersect(
    in_features=["GaragePoints_Buffer150m", r"C:\Gis class\Homework4\Campus.gdb\Structures"],
    out_feature_class=intersection_output
)

print("Intersection created:", intersection_output)


Intersection created: GarageBuildingIntersection


In [6]:
# Export attribute table to CSV
out_csv = r"C:\Gis class\Homework4\garage_building_intersection.csv"

arcpy.conversion.TableToTable(
    in_rows="GarageBuildingIntersection",
    out_path=r"C:\Gis class\Homework4",
    out_name="garage_building_intersection.csv"
)

print("CSV saved to:", out_csv)


CSV saved to: C:\Gis class\Homework4\garage_building_intersection.csv


In [7]:
import arcpy

print("GaragePoints count:",
      arcpy.management.GetCount("GaragePoints"))

print("GaragePoints_Buffer150m count:",
      arcpy.management.GetCount("GaragePoints_Buffer150m"))

print("GarageBuildingIntersection count:",
      arcpy.management.GetCount("GarageBuildingIntersection"))


GaragePoints count: 6
GaragePoints_Buffer150m count: 6
GarageBuildingIntersection count: 62


In [8]:
desc = arcpy.Describe("GaragePoints")
print(desc.spatialReference.name)
print(desc.extent)


GCS_WGS_1984
-96.34426 30.6084900000001 -96.3336199999999 30.61924 NaN NaN NaN NaN


In [9]:
# Project GaragePoints to UTM Zone 14N (meters)
arcpy.management.Project(
    in_dataset="GaragePoints",
    out_dataset="GaragePoints_UTM14N",
    out_coor_system=arcpy.SpatialReference(26914)  # NAD 1983 UTM Zone 14N
)

print("Projected to UTM")


Projected to UTM


In [10]:
arcpy.management.Delete("GaragePoints_Buffer150m")
print("Old buffer deleted")


Old buffer deleted


In [11]:
arcpy.analysis.Buffer(
    in_features="GaragePoints_UTM14N",
    out_feature_class="GaragePoints_Buffer150m",
    buffer_distance_or_field="150 Meters"
)

print("New buffer created")


New buffer created


In [12]:
arcpy.analysis.Intersect(
    in_features=["GaragePoints_Buffer150m", "Structures"],
    out_feature_class="GarageBuildingIntersection"
)

print("Intersection recreated")


<class 'arcgisscripting.ExecuteError'>: Failed to execute. Parameters are not valid.
ERROR 000732: Input Features: Dataset Structures does not exist or is not supported
WARNING 000725: Output Feature Class: Dataset C:\Gis class\Homework4\HWK4_output.gdb\GarageBuildingIntersection already exists.
Failed to execute (Intersect).


In [13]:
import arcpy

arcpy.env.overwriteOutput = True

# FULL PATHS (edit only if your folder name is different)
buffer_fc = r"C:\Gis class\Homework4\HWK4_output.gdb\GaragePoints_Buffer150m"
structures_fc = r"C:\Gis class\Homework4\Campus.gdb\Structures"
out_fc = r"C:\Gis class\Homework4\HWK4_output.gdb\GarageBuildingIntersection"

# Check inputs exist
print("Buffer exists:", arcpy.Exists(buffer_fc))
print("Structures exists:", arcpy.Exists(structures_fc))

# If output already exists, delete it
if arcpy.Exists(out_fc):
    arcpy.management.Delete(out_fc)

# Run intersect
arcpy.analysis.Intersect([buffer_fc, structures_fc], out_fc)

print("✅ Intersection recreated:", out_fc)


Buffer exists: True
Structures exists: True
✅ Intersection recreated: C:\Gis class\Homework4\HWK4_output.gdb\GarageBuildingIntersection


In [14]:
import arcpy

arcpy.conversion.TableToTable(
    in_rows="GarageBuildingIntersection",
    out_path=r"C:\Gis class\Homework4",
    out_name="garage_building_intersection.csv"
)

print("Saved CSV")


Saved CSV
